In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from math import ceil


In [2]:
# Définir les chemins de fichiers
corpus_file = r'D:\etude\M2\RI\nfcorpus\nfcorpus\corpus.jsonl'
queries_file = r'D:\etude\M2\RI\nfcorpus\nfcorpus\queries.jsonl'

# Charger les corpus
corpus_df = pd.read_json(corpus_file, lines=True)
queries_df = pd.read_json(queries_file, lines=True)


In [3]:
# Affichage du nombre de documents et de requêtes
print(f"Nombre de documents : {len(corpus_df)}")
print(f"Nombre de requêtes : {len(queries_df)}")

# Charger les stopwords
stop_words = set(stopwords.words('english'))



Nombre de documents : 3633
Nombre de requêtes : 3237


In [4]:
def tokenize_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read().lower()  # Conversion en minuscules
    
    # Suppression de la ponctuation
    content = re.sub(r'[^\w\s]', '', content)
    tokens = word_tokenize(content)  # Tokenisation
    
    # Filtrage des stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Enregistrement des tokens filtrés 
    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write('\n'.join(filtered_tokens))
    
    # Affichage des tokens filtrés
    print(f"Tokens après filtrage (stopwords) pour {input_file}: {filtered_tokens[:10]}...")  # Afficher les 10 premiers tokens



In [5]:
# Fonction de stemming
stemmer = PorterStemmer()

# Fonction pour effectuer le stemming et enregistrer le résultat
def stemming_tokens(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        tokens = f.read().splitlines()  # Lecture des tokens ligne par ligne
    
    # Application du stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Enregistrement des tokens stemmés dans un fichier de sortie
    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write('\n'.join(stemmed_tokens))
    
    # Affichage des résultats
    print(f"Tokens après stemming pour {input_file}: {stemmed_tokens[:10]}...")  # Afficher les 10 premiers tokens


In [6]:
# Fonction de lemmatisation
lemmatizer = WordNetLemmatizer()

# Fonction pour lemmatiser les tokens d'un fichier et enregistrer le résultat
def lemmatize_tokens(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        tokens = f.read().splitlines()  # Lecture des tokens ligne par ligne
    
    # Application de la lemmatisation
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Enregistrement des tokens lemmatisés dans un fichier de sortie
    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write('\n'.join(lemmatized_tokens))
    
    # Affichage des résultats
    print(f"Tokens après lemmatisation pour {input_file}: {lemmatized_tokens[:10]}...")  # Afficher les 10 premiers tokens



In [7]:
def calculate_tfidf(corpus_file, output_excel_file):
    with open(corpus_file, 'r', encoding='utf-8') as f:
        documents = f.read().splitlines()  # Lire les documents

    # Calcul du TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    
    # Conversion de la matrice TF-IDF en DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), 
                            columns=vectorizer.get_feature_names_out(),  # Les termes dans les colonnes
                            index=[f"Document {i+1}" for i in range(len(documents))])  # Les documents dans les lignes

    # Découper les colonnes en plusieurs groupes si nécessaire
    num_cols = tfidf_df.shape[1]
    max_cols_per_sheet = 16384  # Limite des colonnes par feuille Excel
    num_sheets = ceil(num_cols / max_cols_per_sheet)

    # Écrire dans un fichier Excel avec plusieurs feuilles si nécessaire
    with pd.ExcelWriter(output_excel_file) as writer:
        for i in range(num_sheets):
            start_col = i * max_cols_per_sheet
            end_col = min((i + 1) * max_cols_per_sheet, num_cols)
            sheet_df = tfidf_df.iloc[:, start_col:end_col]
            sheet_name = f"Sheet{i+1}"
            sheet_df.to_excel(writer, sheet_name=sheet_name)

    # Affichage du résultat dans la console (optionnel)
    print(f"TF-IDF matrix saved to {output_excel_file}")
    print(tfidf_df.head())  # Affiche les 5 premières lignes pour vérification


In [8]:
vectorizer = TfidfVectorizer(max_df=0.85, min_df=2)  # Ignore les termes apparaissant dans plus de 85% des documents, ou dans moins de 2 documents


In [ ]:
# Calculer le TF-IDF et l'exporter dans un fichier Excel lisible
calculate_tfidf('corpus.txt', 'tfidf_results.xlsx')


In [74]:
# Appel des fonctions pour tokeniser, stemmer et lemmatiser
tokenize_file('corpus.txt', 'corpus_tokens.txt')
tokenize_file('queries.txt', 'queries_tokens.txt')

Tokens après filtrage (stopwords) pour corpus.txt: ['med10statin', 'use', 'breast', 'cancer', 'survival', 'nationwide', 'cohort', 'study', 'finlandrecent', 'studies']...
Tokens après filtrage (stopwords) pour queries.txt: ['plain3breast', 'cancer', 'cells', 'feed', 'cholesterolurl', 'httpnutritionfactsorg20150714breastcancercellsfeedoncholesterol', 'plain4using', 'diet', 'treat', 'asthma']...


In [75]:
stemming_tokens('corpus_tokens.txt', 'corpus_stemmed_tokens.txt')
stemming_tokens('queries_tokens.txt', 'queries_stemmed_tokens.txt')


Tokens après stemming pour corpus_tokens.txt: ['med10statin', 'use', 'breast', 'cancer', 'surviv', 'nationwid', 'cohort', 'studi', 'finlandrec', 'studi']...
Tokens après stemming pour queries_tokens.txt: ['plain3breast', 'cancer', 'cell', 'feed', 'cholesterolurl', 'httpnutritionfactsorg20150714breastcancercellsfeedoncholesterol', 'plain4us', 'diet', 'treat', 'asthma']...


In [76]:
lemmatize_tokens('corpus_tokens.txt', 'corpus_lemmatized_tokens.txt')
lemmatize_tokens('queries_tokens.txt', 'queries_lemmatized_tokens.txt')


Tokens après lemmatisation pour corpus_tokens.txt: ['med10statin', 'use', 'breast', 'cancer', 'survival', 'nationwide', 'cohort', 'study', 'finlandrecent', 'study']...
Tokens après lemmatisation pour queries_tokens.txt: ['plain3breast', 'cancer', 'cell', 'feed', 'cholesterolurl', 'httpnutritionfactsorg20150714breastcancercellsfeedoncholesterol', 'plain4using', 'diet', 'treat', 'asthma']...


In [77]:
# Calculer le TF-IDF et afficher les résultats
calculate_tfidf('corpus.txt')


TF-IDF matrix:
         00  000  000006073  00001  0001  0002  0003  0004  0005  0006  ...  \
0  0.000000  0.0        0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1  0.047673  0.0        0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2  0.000000  0.0        0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3  0.000000  0.0        0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4  0.000000  0.0        0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

    μl   μm  μmol   μu  σ7pbdes  σoh  σpbde5  σttr   χ2   ⁶m  
0  0.0  0.0   0.0  0.0      0.0  0.0     0.0   0.0  0.0  0.0  
1  0.0  0.0   0.0  0.0      0.0  0.0     0.0   0.0  0.0  0.0  
2  0.0  0.0   0.0  0.0      0.0  0.0     0.0   0.0  0.0  0.0  
3  0.0  0.0   0.0  0.0      0.0  0.0     0.0   0.0  0.0  0.0  
4  0.0  0.0   0.0  0.0      0.0  0.0     0.0   0.0  0.0  0.0  

[5 rows x 33161 columns]
